# 使用LLaMA Factory微调Qwen3模型

[LLaMA Factory](https://github.com/hiyouga/LLaMA-Factory)是一款开源低代码大模型微调框架，集成了业界最广泛使用的微调技术，支持通过Web UI界面零代码微调大模型，目前已经成为开源社区内最受欢迎的微调框架，GitHub星标超过2万。
本教程将基于开源的Qwen3 8B模型，介绍如何使用PAI平台及LLaMA Factory训练框架完成中文医疗领域的微调和评估。 

## 运行环境要求

- GPU推荐使用24GB显存的A10（`ecs.gn7i-c8g1.2xlarge`）或更高配置
- 镜像选择DSW官方镜像`pytorch:2.6.0-gpu-py311-cu124-ubuntu22.04`

## 系统环境检查

In [1]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                              |
|:-------------|:----------------------------------------------------------------------------------|
| 操作系统     | Linux 5.10.134-18.0.4.lifsea8.x86_64                                              |
| CPU 信息     | Intel(R) Xeon(R) Platinum 8369B CPU @ 2.90GHz (4 physical cores, 8 logical cores) |
| 内存信息     | 29.37 GB (Available: 27.26 GB)                                                    |
| GPU 信息     | NVIDIA A10 (24564 MiB)                                                            |
| CUDA 信息    | 12.4                                                                              |
| Python 版本  | 3.11.14                                                                           |
| Conda 版本   | Conda not found                                               

## 检查软硬件依赖

| 必需项       | 至少   | 推荐   |
| ------------ | ------ | ------ |
| python       | 3.9    | 3.10   |
| torch        | 2.0.0  | 2.6.0  |
| torchvision  | 0.15.0 | 0.21.0 |
| transformers | 4.45.0 | 4.50.0 |
| datasets     | 2.16.0 | 3.2.0  |
| accelerate   | 0.34.0 | 1.2.1  |
| peft         | 0.14.0 | 0.15.1 |
| trl          | 0.8.6  | 0.9.6  |

| 可选项       | 至少   | 推荐   |
| ------------ | ------ | ------ |
| CUDA         | 11.6   | 12.2   |
| deepspeed    | 0.10.0 | 0.16.4 |
| bitsandbytes | 0.39.0 | 0.43.1 |
| vllm         | 0.4.3  | 0.8.2  |
| flash-attn   | 2.5.6  | 2.7.2  |

### 硬件依赖

\* *估算值*

| 方法                            | 精度 | 7B    | 14B   | 30B   | 70B    | `x`B    |
| ------------------------------- | ---- | ----- | ----- | ----- | ------ | ------- |
| Full (`bf16` or `fp16`)         | 32   | 120GB | 240GB | 600GB | 1200GB | `18x`GB |
| Full (`pure_bf16`)              | 16   | 60GB  | 120GB | 300GB | 600GB  | `8x`GB  |
| Freeze/LoRA/GaLore/APOLLO/BAdam | 16   | 16GB  | 32GB  | 64GB  | 160GB  | `2x`GB  |
| QLoRA                           | 8    | 10GB  | 20GB  | 40GB  | 80GB   | `x`GB   |
| QLoRA                           | 4    | 6GB   | 12GB  | 24GB  | 48GB   | `x/2`GB |
| QLoRA                           | 2    | 4GB   | 8GB   | 16GB  | 24GB   | `x/4`GB |

In [2]:
import sys
import platform
import importlib.metadata
from packaging import version
import torch

# ================= 您的最新配置要求 (已再次更新) =================
REQUIRED_PACKAGES = {
    "python":       ("3.9",    "3.10"),
    "torch":        ("2.0.0",  "2.6.0"),
    "torchvision":  ("0.15.0", "0.21.0"),
    "transformers": ("4.45.0", "4.50.0"),
    "datasets":     ("2.16.0", "3.2.0"),
    "accelerate":   ("0.34.0", "1.2.1"),
    "peft":         ("0.14.0", "0.15.1"),
    "trl":          ("0.8.6",  "0.9.6"),
}

OPTIONAL_PACKAGES = {
    "cuda":         ("11.6",   "12.2"),
    "deepspeed":    ("0.10.0", "0.16.4"),
    "bitsandbytes": ("0.39.0", "0.43.1"),
    "vllm":         ("0.4.3",  "0.8.2"),
    "flash-attn":   ("2.5.6",  "2.7.2"),
}

# ================= 检测逻辑 =================
def get_package_version(package_name):
    try:
        if package_name == "python": return platform.python_version()
        elif package_name == "cuda": return torch.version.cuda if torch.cuda.is_available() else None
        elif package_name == "flash-attn": return importlib.metadata.version("flash_attn")
        else: return importlib.metadata.version(package_name)
    except importlib.metadata.PackageNotFoundError: return None

def check_requirement(name, constraints, is_optional=False):
    min_v, rec_v = constraints
    current_v = get_package_version(name)
    
    # 定义列宽以保持对齐
    col_name = 15
    col_curr = 15
    col_req = 10
    
    if current_v is None:
        status, msg, color = ("⚪", "未安装 (可选)", "\033[90m") if is_optional else ("❌", "未安装 !!", "\033[91m")
    else:
        try:
            curr_p, min_p, rec_p = version.parse(current_v), version.parse(min_v), version.parse(rec_v)
            if curr_p >= rec_p:   status, msg, color = ("✅", f"完美 ({current_v})", "\033[92m") # Green
            elif curr_p >= min_p: status, msg, color = ("⚠️", f"达标 ({current_v})", "\033[93m") # Yellow
            else:                 status, msg, color = ("❌", f"过低 ({current_v})", "\033[91m") # Red
        except Exception as e:
            status, msg, color = ("❓", f"解析错误", "\033[93m")

    print(f"{color}{status} {name:<{col_name}} | 当前: {msg:<{col_curr}} | 必需: >={min_v:<{col_req}} | 推荐: >={rec_v}\033[0m")

def check_gpu():
    print("\n[ 硬件简测 ]")
    if torch.cuda.is_available():
        device_count = torch.cuda.device_count()
        print(f"✅ 检测到 {device_count} 个 GPU 设备:")
        for i in range(device_count):
            gpu_name = torch.cuda.get_device_name(i)
            # 转换为 GB
            total_mem = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"   - GPU {i}: {gpu_name} ({total_mem:.1f} GB VRAM)")
    else:
        print("⚪ 未检测到可用 GPU (CUDA不可用)")

print("="*75)
print(f"🛠️  环境依赖检测报告 (标准已再次更新)")
print("="*75)
print("[ 必需项 ]")
for k, v in REQUIRED_PACKAGES.items(): check_requirement(k, v)
print("\n[ 可选项 ]")
for k, v in OPTIONAL_PACKAGES.items(): check_requirement(k, v, True)
check_gpu()
print("="*75)

🛠️  环境依赖检测报告 (标准已再次更新)
[ 必需项 ]
✅ python          | 当前: 完美 (3.11.14)    | 必需: >=3.9        | 推荐: >=3.10
✅ torch           | 当前: 完美 (2.6.0+cu124) | 必需: >=2.0.0      | 推荐: >=2.6.0
✅ torchvision     | 当前: 完美 (0.21.0+cu124) | 必需: >=0.15.0     | 推荐: >=0.21.0
✅ transformers    | 当前: 完美 (4.52.4)     | 必需: >=4.45.0     | 推荐: >=4.50.0
✅ datasets        | 当前: 完美 (3.6.0)      | 必需: >=2.16.0     | 推荐: >=3.2.0
✅ accelerate      | 当前: 完美 (1.7.0)      | 必需: >=0.34.0     | 推荐: >=1.2.1
✅ peft            | 当前: 完美 (0.15.2)     | 必需: >=0.14.0     | 推荐: >=0.15.1
✅ trl             | 当前: 完美 (0.9.6)      | 必需: >=0.8.6      | 推荐: >=0.9.6

[ 可选项 ]
✅ cuda            | 当前: 完美 (12.4)       | 必需: >=11.6       | 推荐: >=12.2
⚪ deepspeed       | 当前: 未安装 (可选)        | 必需: >=0.10.0     | 推荐: >=0.16.4
✅ bitsandbytes    | 当前: 完美 (0.48.2)     | 必需: >=0.39.0     | 推荐: >=0.43.1
⚪ vllm            | 当前: 未安装 (可选)        | 必需: >=0.4.3      | 推荐: >=0.8.2
⚪ flash-attn      | 当前: 未安装 (可选)        | 必需: >=2.5.6      | 推荐: >=2.7.2

[ 硬件

## 1. 安装LLaMA Factory


首先，拉取LLaMA-Factory项目到DSW实例。

In [3]:
import os
import shutil

# 定义仓库文件夹名称
repo_name = "LLaMA-Factory"

# 1. 判断文件夹是否存在
if os.path.exists(repo_name):
    # 获取用户输入
    user_input = input(f"⚠️ 检测到旧目录 '{repo_name}'，是否删除？(输入 y 确认删除，其他键取消): ")
    
    # 判断用户输入 (支持 y 或 Y)
    if user_input.lower() == 'y':
        print(f"正在删除 '{repo_name}'...")
        try:
            shutil.rmtree(repo_name) # 执行实际删除
            print("🗑️ 旧目录已清除！")
        except Exception as e:
            print(f"❌ 删除失败: {e}")
    else:
        print("🚫 操作已取消，保留旧目录。")
else:
    print(f"✅ 目录 '{repo_name}' 不存在，准备下载仓库...")
    # 克隆仓库
    !git clone -b v0.9.3 https://github.com/hiyouga/LLaMA-Factory.git

⚠️ 检测到旧目录 'LLaMA-Factory'，是否删除？(输入 y 确认删除，其他键取消):  n


🚫 操作已取消，保留旧目录。


接着，我们安装LLaMA-Factory依赖环境。

In [4]:
# 1. 【关键】使用 %cd 进入目录 (不要用 !cd)
%cd /mnt/workspace/LLaMA-Factory

# 2. 去掉无关依赖
!pip install --upgrade pip
!pip install modelscope>=1.11.0
!pip install bitsandbytes>=0.39.0

# 3. 安装项目依赖
# 这会自动安装 v0.9.3 所需的其他库
!pip install -e ".[torch,metrics]"

/mnt/workspace/LLaMA-Factory
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Obtaining file:///mnt/workspace/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.3-0.editable-py3-none-any.whl size=27459 sha256=fa3429241150a8b0052e075455dcc92babda44c8ff52fc283ad590497c0a725a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ptpzmd_k/wheels/18/63/ec/7d8dd1bfb5f04a52ee923c20e91cc54ec06a170b202ec6a30d
Successfully built llamafactory
  Attempting uninstall: llamafactory
    Found existing installation: llamafactory 0.9.3
    Uninstalling llamafactory-0.9.3:
      Successfully uninstalled llamafactory-0.9.3


### 检验依赖是否满足

In [5]:

import sys
import platform
import importlib.metadata
from packaging import version
import torch

# ================= 您的最新配置要求 (已再次更新) =================
REQUIRED_PACKAGES = {
    "python":       ("3.9",    "3.10"),
    "torch":        ("2.0.0",  "2.6.0"),
    "torchvision":  ("0.15.0", "0.21.0"),
    "transformers": ("4.45.0", "4.50.0"),
    "datasets":     ("2.16.0", "3.2.0"),
    "accelerate":   ("0.34.0", "1.2.1"),
    "peft":         ("0.14.0", "0.15.1"),
    "trl":          ("0.8.6",  "0.9.6"),
}

OPTIONAL_PACKAGES = {
    "cuda":         ("11.6",   "12.2"),
    "deepspeed":    ("0.10.0", "0.16.4"),
    "bitsandbytes": ("0.39.0", "0.43.1"),
    "vllm":         ("0.4.3",  "0.8.2"),
    "flash-attn":   ("2.5.6",  "2.7.2"),
}

# ================= 检测逻辑 =================
def get_package_version(package_name):
    try:
        if package_name == "python": return platform.python_version()
        elif package_name == "cuda": return torch.version.cuda if torch.cuda.is_available() else None
        elif package_name == "flash-attn": return importlib.metadata.version("flash_attn")
        else: return importlib.metadata.version(package_name)
    except importlib.metadata.PackageNotFoundError: return None

def check_requirement(name, constraints, is_optional=False):
    min_v, rec_v = constraints
    current_v = get_package_version(name)
    
    # 定义列宽以保持对齐
    col_name = 15
    col_curr = 15
    col_req = 10
    
    if current_v is None:
        status, msg, color = ("⚪", "未安装 (可选)", "\033[90m") if is_optional else ("❌", "未安装 !!", "\033[91m")
    else:
        try:
            curr_p, min_p, rec_p = version.parse(current_v), version.parse(min_v), version.parse(rec_v)
            if curr_p >= rec_p:   status, msg, color = ("✅", f"完美 ({current_v})", "\033[92m") # Green
            elif curr_p >= min_p: status, msg, color = ("⚠️", f"达标 ({current_v})", "\033[93m") # Yellow
            else:                 status, msg, color = ("❌", f"过低 ({current_v})", "\033[91m") # Red
        except Exception as e:
            status, msg, color = ("❓", f"解析错误", "\033[93m")

    print(f"{color}{status} {name:<{col_name}} | 当前: {msg:<{col_curr}} | 必需: >={min_v:<{col_req}} | 推荐: >={rec_v}\033[0m")

def check_gpu():
    print("\n[ 硬件简测 ]")
    if torch.cuda.is_available():
        device_count = torch.cuda.device_count()
        print(f"✅ 检测到 {device_count} 个 GPU 设备:")
        for i in range(device_count):
            gpu_name = torch.cuda.get_device_name(i)
            # 转换为 GB
            total_mem = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"   - GPU {i}: {gpu_name} ({total_mem:.1f} GB VRAM)")
    else:
        print("⚪ 未检测到可用 GPU (CUDA不可用)")

print("="*75)
print(f"🛠️  环境依赖检测报告 (标准已再次更新)")
print("="*75)
print("[ 必需项 ]")
for k, v in REQUIRED_PACKAGES.items(): check_requirement(k, v)
print("\n[ 可选项 ]")
for k, v in OPTIONAL_PACKAGES.items(): check_requirement(k, v, True)
check_gpu()
print("="*75)

🛠️  环境依赖检测报告 (标准已再次更新)
[ 必需项 ]
✅ python          | 当前: 完美 (3.11.14)    | 必需: >=3.9        | 推荐: >=3.10
✅ torch           | 当前: 完美 (2.6.0+cu124) | 必需: >=2.0.0      | 推荐: >=2.6.0
✅ torchvision     | 当前: 完美 (0.21.0+cu124) | 必需: >=0.15.0     | 推荐: >=0.21.0
✅ transformers    | 当前: 完美 (4.52.4)     | 必需: >=4.45.0     | 推荐: >=4.50.0
✅ datasets        | 当前: 完美 (3.6.0)      | 必需: >=2.16.0     | 推荐: >=3.2.0
✅ accelerate      | 当前: 完美 (1.7.0)      | 必需: >=0.34.0     | 推荐: >=1.2.1
✅ peft            | 当前: 完美 (0.15.2)     | 必需: >=0.14.0     | 推荐: >=0.15.1
✅ trl             | 当前: 完美 (0.9.6)      | 必需: >=0.8.6      | 推荐: >=0.9.6

[ 可选项 ]
✅ cuda            | 当前: 完美 (12.4)       | 必需: >=11.6       | 推荐: >=12.2
⚪ deepspeed       | 当前: 未安装 (可选)        | 必需: >=0.10.0     | 推荐: >=0.16.4
✅ bitsandbytes    | 当前: 完美 (0.48.2)     | 必需: >=0.39.0     | 推荐: >=0.43.1
⚪ vllm            | 当前: 未安装 (可选)        | 必需: >=0.4.3      | 推荐: >=0.8.2
⚪ flash-attn      | 当前: 未安装 (可选)        | 必需: >=2.5.6      | 推荐: >=2.7.2

[ 硬件

运行如下命令，如果显示llamafactory-cli的版本，则表示安装成功。

In [6]:
!llamafactory-cli version

----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.9.3                |
|                                                        |
| Project page: https://github.com/hiyouga/LLaMA-Factory |
----------------------------------------------------------


## 2. 下载数据集

LLaMA-Factory项目内置了丰富的数据集，放在了`data`目录下。您可以跳过本步骤，直接使用内置数据集。您也可以准备自定义数据集，将数据处理为框架特定的格式，放在`data`下，并且修改`dataset_info.json`文件。

本教程准备了一份多轮对话数据集，运行下述命令下载数据。

In [7]:
!wget https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/release/tutorials/llama_factory/data.zip
!mv data rawdata && unzip data.zip -d data

--2025-12-04 18:27:10--  https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/release/tutorials/llama_factory/data.zip
Resolving atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)... 47.101.88.43
Connecting to atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)|47.101.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 402043 (393K) [application/zip]
Saving to: ‘data.zip.2’

data.zip.2          100%[===================>] 392.62K  --.-KB/s    in 0.1s    

2025-12-04 18:27:10 (3.73 MB/s) - ‘data.zip.2’ saved [402043/402043]

mv: cannot move 'data' to 'rawdata/data': Directory not empty


上传医疗数据集，将jsonl文件拖拽到`/mnt/workspace`文件下。
```
root@dsw-523480-fd7f95d6d-cgjwf:/mnt/workspace# pwd
/mnt/workspace
root@dsw-523480-fd7f95d6d-cgjwf:/mnt/workspace# ll
total 63916
drwxr-xr-x  7 root root     4096 Dec  3 16:32  ./
drwxr-xr-x  1 root root     4096 Dec  3 15:06  ../
drwxr-xr-x  2 root root     4096 Dec  3 15:19  _html/
drwxr-xr-x 16 root root     4096 Dec  3 15:59  LLaMA-Factory/
-rw-r--r--  1 root root    87581 Dec  3 16:32  llama-factory-in-ali-pai.ipynb
drwxr-xr-x  3 root root     4096 Dec  3 15:06  .local/
-rw-r--r--  1 root root 65325060 Dec  3 16:21  medical_o1_sft_Chinese_alpaca_cot.jsonl
drwxr-xr-x  2 root root     4096 Dec  3 15:10  .virtual_documents/
```


修改`/mnt/workspace`文件夹下“dataset_json.info”文件，在json文件开始位置添加如下所示脚本。
```
"medical_sft": {
    "file_name": "/workspace/LLaMA-Factory/data/medical_o1_sft_Chinese_alpaca_cot.jsonl",
    "columns": {
      "prompt": "prompt",
      "response": "response"
    }
  }
```

“dataset_json.info”文件最终如下
```
{
  "medical_sft": {
    "file_name": "/mnt/workspace/medical_o1_sft_Chinese_alpaca_cot.jsonl",
    "columns": {
      "prompt": "prompt",
      "response": "response"
    }
  },
  "train": {
    "file_name": "train.json",
    "formatting": "sharegpt"
  },
  "eval": {
    "file_name": "eval.json",
    "formatting": "sharegpt"
  }
}
```

In [8]:
import json
import os

# 1. 你的数据集路径
jsonl_file = "/mnt/workspace/medical_o1_sft_Chinese_alpaca_cot.jsonl"
# 2. LLaMA-Factory 的配置文件路径
config_file = "/mnt/workspace/LLaMA-Factory/data/dataset_info.json"

# 3. 要添加的配置内容 (Python字典格式，不需要手动管逗号)
new_data = {
    "medical_sft": {
        "file_name": jsonl_file,
        "columns": {
            "prompt": "prompt",
            "response": "response"
        }
    }
}

# 4. 执行自动写入
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        content = json.load(f)
        
    # 更新数据（程序会自动处理逗号分割）
    content.update(new_data)
    
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(content, f, indent=2, ensure_ascii=False)
        
    print(f"✅ 成功！已自动添加逗号并注册 'medical_sft' 到：\n{config_file}")

except FileNotFoundError:
    print("❌ 错误：找不到 LLaMA-Factory 的配置文件，请检查目录结构。")
except json.JSONDecodeError:
    print("❌ 错误：原配置文件格式损坏（可能之前手动改坏了），请还原文件。")

✅ 成功！已自动添加逗号并注册 'medical_sft' 到：
/mnt/workspace/LLaMA-Factory/data/dataset_info.json


## 3. 模型微调
### 3.1 启动Web UI

做好前序准备工作后，直接运行下述命令就可以启动Web UI。这里用到的环境变量解释如下：
- `USE_MODELSCOPE_HUB`设为1，表示模型来源是ModelScope。使用HuggingFace模型可能会有网络问题。

点击返回的URL地址`http://0.0.0.0:7860`，进入Web UI页面。

In [ ]:
!export USE_MODELSCOPE_HUB=1 && \
llamafactory-cli webui

Visit http://ip:port for Web UI, e.g., http://127.0.0.1:7860
* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
[WARNING|2025-12-04 19:05:54] llamafactory.hparams.parser:148 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2025-12-04 19:05:54] llamafactory.hparams.parser:406 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.bfloat16
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >>

### 3.2 配置参数
结合 **A10 (24G)** 显卡与 **Qwen3-8B** 的环境，以下是针对配置界面的详细参数解读。这些设置共同构成了 **QLoRA** 微调方案，是在单卡环境下处理医疗推理（Reasoning）长文本数据的最佳实践。

![image-20251204185719305](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041857405.png)

#### 3.2.1 模型路径与名称配置 (Model Configuration)
* **参数项**：`模型名称` & `模型路径`
* **当前值**：
    * 名称：`Qwen3-8B-Instruct`
    * 路径：`/mnt/workspace/Qwen3-8B`
* **配置作用**：
    * **模型路径**：这是“地基”。程序会从 `/mnt/workspace/Qwen3-8B` 读取模型权重的物理文件。
    * **模型名称**：这是“标识”。LLaMA-Factory 会根据这个名称自动匹配默认的超参数和模板。
* **硬件影响**：A10 的 24G 显存加载 8B 模型毫无压力，这一步是基础。

模型路径使用本地路径，如：`/mnt/workspace/Qwen3-8B`。（考虑公网有时候下载模型慢，提供给大家准备好了离线模型压缩包，可以解压到`/mnt/workspace/Qwen3-8B`）

其中`/mnt/workspace/Qwen3-8B`内容如下，遇到出错，请核对文件夹是不是少内容

```bash
root@dsw-523480-6d6bb75cf6-kz46d:/mnt/workspace/Qwen3-8B# ll
total 16013364
drwxr-xr-x  2 root root       4096 Dec  4 17:47 ./
drwxr-xr-x 10 root root       4096 Dec  4 18:38 ../
-rw-r--r--  1 root root        757 Dec  4 17:02 config.json
-rw-r--r--  1 root root         73 Dec  4 17:02 configuration.json
-rw-r--r--  1 root root        251 Dec  4 17:02 generation_config.json
-rw-r--r--  1 root root       2175 Dec  4 17:02 .gitattributes
-rw-r--r--  1 root root      11544 Dec  4 17:02 LICENSE
-rw-r--r--  1 root root    1823241 Dec  4 17:02 merges.txt
-rw-r--r--  1 root root 3996250744 Dec  4 17:41 model-00001-of-00005.safetensors
-rw-r--r--  1 root root 3993160032 Dec  4 17:45 model-00002-of-00005.safetensors
-rw-r--r--  1 root root 3959604768 Dec  4 17:47 model-00003-of-00005.safetensors
-rw-r--r--  1 root root 3187841392 Dec  4 17:43 model-00004-of-00005.safetensors
-rw-r--r--  1 root root 1244659840 Dec  4 17:14 model-00005-of-00005.safetensors
-rw-r--r--  1 root root      33284 Dec  4 17:02 model.safetensors.index.json
-rw-r--r--  1 root root      17012 Dec  4 17:02 README.md
-rw-r--r--  1 root root       9971 Dec  4 17:02 tokenizer_config.json
-rw-r--r--  1 root root   11422654 Dec  4 17:02 tokenizer.json
-rw-r--r--  1 root root    2776833 Dec  4 17:02 vocab.json
```

这是典型的 Hugging Face `transformers` 格式的模型目录结构（Qwen 系列）。

这个目录包含了 **模型架构配置**、**分词器（Tokenizer）** 和 **模型权重（Weights）** 三大核心部分。

以下是逐行注释解释：

```bash
# ----------------- 核心配置区 -----------------
-rw-r--r--  1 root root        757 Dec  4 17:02 config.json             # 【最关键】模型架构配置文件。定义了模型有多少层、隐藏层大小、注意力头数等。代码通过它来构建模型骨架。
-rw-r--r--  1 root root         73 Dec  4 17:02 configuration.json      # 辅助配置文件（通常较少见，可能是特定框架生成的元数据，通常 config.json 是主文件）。
-rw-r--r--  1 root root        251 Dec  4 17:02 generation_config.json  # 推理参数配置。定义了模型生成文本时的默认行为（如 max_length, temperature, top_p, eos_token_id 等）。

# ----------------- 版本控制与协议 -----------------
-rw-r--r--  1 root root       2175 Dec  4 17:02 .gitattributes          # Git LFS 配置文件。告诉 git 哪些文件是大文件（如 .safetensors），需要用 LFS 方式下载。
-rw-r--r--  1 root root      11544 Dec  4 17:02 LICENSE                 # 模型的开源许可证文件（如 Apache 2.0 或 Qwen Research License），规定了你能否商用。

# ----------------- 分词器 (Tokenizer) 词表区 -----------------
# 作用：将人类文字转换成数字 ID，反之亦然
-rw-r--r--  1 root root    1823241 Dec  4 17:02 merges.txt              # BPE 合并规则文件。记录了哪些字符组合可以合并成一个 Token（常见于 GPT 类模型）。
-rw-r--r--  1 root root    2776833 Dec  4 17:02 vocab.json              # 词表文件。一个巨大的字典，记录了所有 Token 字符串到数字 ID 的映射关系。
-rw-r--r--  1 root root   11422654 Dec  4 17:02 tokenizer.json          # 【重要】Fast Tokenizer 的完整定义文件。包含了分词的所有逻辑，加载速度比 Python 代码快。
-rw-r--r--  1 root root       9971 Dec  4 17:02 tokenizer_config.json   # 分词器设置文件。定义了特殊 Token（如 <|endoftext|>），以及使用哪个 Tokenizer 类。

# ----------------- 模型权重区 (Weights) -----------------
# 作用：模型的“大脑”，存储了训练好的参数（矩阵数值）
# 由于 8B 模型很大（约 15GB+），所以被切分成了 5 个文件（Sharding）
-rw-r--r--  1 root root 3996250744 Dec  4 17:41 model-00001-of-00005.safetensors # 权重分片 1/5
-rw-r--r--  1 root root 3993160032 Dec  4 17:45 model-00002-of-00005.safetensors # 权重分片 2/5
-rw-r--r--  1 root root 3959604768 Dec  4 17:47 model-00003-of-00005.safetensors # 权重分片 3/5
-rw-r--r--  1 root root 3187841392 Dec  4 17:43 model-00004-of-00005.safetensors # 权重分片 4/5
-rw-r--r--  1 root root 1244659840 Dec  4 17:14 model-00005-of-00005.safetensors # 权重分片 5/5

# ----------------- 权重索引与说明 -----------------
-rw-r--r--  1 root root      33284 Dec  4 17:02 model.safetensors.index.json     # 权重索引映射表。因为权重被切分了，这个文件告诉加载器：“第 X 层的参数在第 Y 个 safetensors 文件里”。
-rw-r--r--  1 root root      17012 Dec  4 17:02 README.md               # 模型说明书（Markdown 格式）。通常包含模型介绍、引用方法、使用示例代码等。
```

当你运行 `AutoModelForCausalLM.from_pretrained("这个目录")` 时：

1. **读取 `config.json`**：搭建空的神经网络骨架。
2. **读取 `model.safetensors.index.json`**：查找参数都在哪。
3. **加载 `model-0000\*.safetensors`**：把具体的数字填入骨架中。

当你运行 `AutoTokenizer.from_pretrained("这个目录")` 时：

1. **读取 `tokenizer_config.json`**：确认配置。
2. **优先加载 `tokenizer.json`**：如果失败，才去尝试组合 `vocab.json` 和 `merges.txt`。

#### 3.2.2 微调核心方法 (Finetuning Method)
* **参数项**：`微调方法`
* **当前值**：`lora`
* **配置作用**：
    * 选择 **LoRA** (Low-Rank Adaptation) 技术。它不会修改模型原本的几十亿个参数，而是在模型旁挂载一些小的“适配器”矩阵来训练。
* **为什么不选 full（全量）**：
    * 全量微调 8B 模型需要保存庞大的优化器状态，至少需要 100GB+ 的显存。
    * **A10 (24G) 的唯一出路**：对于单卡 A10，必须使用 LoRA 或 QLoRA 才能跑得起来。

#### 3.2.3 量化设置 (Quantization) —— 关键优化
这一步是将 **LoRA** 升级为 **QLoRA** 的关键，直接决定了您能训练多长的医疗病例。

* **参数项**：`量化等级`
* **当前值**：`4`
* **配置作用**：
    * 将模型权重的精度从 16-bit (半精度) 压缩到 4-bit。
* **对 A10 (24G) 的巨大影响**：
    * **不量化 (16-bit)**：8B 模型仅权重就占用约 **16GB** 显存。此时 A10 只剩 8GB 给训练数据，只要医疗文本稍微长一点（例如包含复杂的病情描述），显存就会瞬间溢出 (OOM)。
    * **4-bit 量化**：模型权重仅占用约 **5.5GB** 显存。
    * **收益**：您空出了近 **18GB** 的显存！这意味着您可以将 `Context Length`（上下文长度）拉得很大（例如 4096 或 8192），这对于 **medical-o1-reasoning** 这种需要长逻辑链推理的数据集至关重要。

* **参数项**：`量化方法`
* **当前值**：`bnb` (bitsandbytes)
* **配置作用**：
    * 这是实现 4-bit 量化的底层加速库。您的环境是 Ubuntu 22.04 + CUDA 12.4，`bnb` 能够完美调用 A10 的硬件加速能力。

#### 3.2.4 数据模板与格式 (Template)
* **参数项**：`对话模板`
* **当前值**：`qwen3` (注意：截图可能需确认为 qwen 或 qwen2.5/3 对应模板，Qwen3 通常兼容 Qwen2.5 模板)
* **配置作用**：
    * 如同“翻译器”，它将您的医疗数据集格式化为 Qwen 模型能听懂的特殊字符结构（例如 `<|im_start|>`）。
* **严重后果**：如果选错（例如选成 llama3），模型会因为看不懂数据结构而输出乱码，或者在训练时 loss 无法下降。务必确保与模型系列一致。

#### 3.2.5 硬件加速策略 (Booster)
* **参数项**：`加速方式`
* **当前值**：`auto`
* **配置作用**：
    * 自动选择最优的注意力机制加速算法，通常会启用 **FlashAttention-2**。
* **环境匹配**：
    * A10 显卡属于 Ampere 架构，完美支持 FlashAttention-2。
    * **效果**：相比不开启加速，训练速度通常能提升 2-3 倍，且能进一步节省显存。保持 `auto` 是最省心的选择。
    

#### 3.2.6 训练阶段 (Training Stage)
* **配置项**：`训练阶段`
* **当前设置**：`Supervised Fine-Tuning` (SFT)
* **详细解释**：
    * **定义**：全称是“有监督微调”。这是目前让大模型适配特定领域（如医疗）最直接、最高效的方法。
    * **原理**：就好比给学生（模型）看大量的“问题+标准答案”的试卷，让它学会照葫芦画瓢。与之相对的还有“预训练（Pre-Training）”（读课本，学通识）和“DPO/RLHF”（学价值观，分辨好坏）。
    * **适用场景**：您现在的目标是让 Qwen3 拥有医疗推理能力，SFT 是标准选择。
    * **环境影响**：SFT 相比预训练极其节省资源。在您的 A10 (24G) 上，配合 LoRA，SFT 运行起来非常轻量。

#### 3.2.7 数据路径与数据集选择 (Data Configuration)
* **配置项**：`数据路径 (Data Path)` & `数据集 (Dataset)`
* **当前设置**：
    * 路径：`data`
    * 数据集：`medical_sft` (截图显示您已选中此项)
* **详细解释 (新手必读)**：
    * **数据路径**：指向 LLaMA-Factory 根目录下的 `data` 文件夹。这里面存放着至关重要的 `dataset_info.json` 配置文件。
    * **数据集选择**：
        * 这里的 `medical_sft` 并不是文件名，而是在 `dataset_info.json` 中注册的一个**标签（Key）**。
        * **关键操作**：您之前提到了 `medical-o1-reasoning-SFT` 数据集。您需要确保已经在 `data/dataset_info.json` 文件中添加了对应的配置，将 `medical_sft` 这个标签指向您实际下载的 JSON 或 Parquet 文件。
        * *错误排查*：如果您在下拉菜单里找不到您的数据集，通常是因为没有在 `dataset_info.json` 里注册，或者文件没放对位置。
* **作用**：告诉程序使用哪一份具体的医疗病历数据来喂给模型。

#### 3.2.8 数据预览功能 (Dataset Preview)
* **配置项**：`预览数据集` (按钮)
* **建议操作**：**强烈建议点击**。
* **详细解释**：
    * **作用**：它会读取前几行数据并展示出来。
    * **新手检查点**：
        1.  **列名匹配**：检查展示的数据中，是否包含“instruction”（指令/问题）、“input”（输入/病历详情，可选）、“output”（输出/诊断结果）这几列。
        2.  **格式正常**：确保没有乱码，且内容确实是您预期的医疗推理数据。
    * **为什么重要**：如果数据格式不对（比如列名写成了 `question` 而不是 `instruction`），模型训练时会找不到输入，导致训练虽然在跑，但什么都没学到（Loss 不下降）。在开始漫长的训练前，花 10 秒钟预览是性价比最高的操作。

数据集使用我们的医疗数据集`medical_sft`。
![image-20251203165018105](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031650258.png)
可以点击「预览数据集」。点击关闭返回训练界面。
![image-20251203165125842](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031651285.png)







设置`学习率`为`1e-4`，`梯度累积`为`2`，有利于模型拟合。如果显卡是`V100`，计算类型保持为`fp16`；如果使用了`A10`，可以更改计算类型为`bf16`。
![image-20251203165304488](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031653595.png)

点击`LoRA参数设置`展开参数列表，设置`LoRA+学习率比例`为`16`，`LoRA+`被证明是比LoRA学习效果更好的算法。在`LoRA作用模块`中填写`all`，即将LoRA层挂载到模型的所有线性层上，提高拟合效果。
![image-20251203165600495](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031656725.png)


#### 3.2.9 学习率 (Learning Rate)

- **配置项**：`学习率`
- **当前值**：`1e-4` (即 0.0001)
- **详细解释**：
  - **作用**：决定了模型“迈步子”的大小。
  - **新手指南**：
    - 对于 **LoRA** 微调，`1e-4` 是一个非常标准且稳健的“黄金初始值”。
    - 如果是全量微调，这个值通常要小得多（如 `1e-5`）。但对于 LoRA，保持 `1e-4` 通常能获得不错的收敛速度。
  - **调整建议**：初次运行建议保持不变。如果发现 Loss（损失值）完全不下降，可以尝试稍微调大；如果 Loss 剧烈震荡，则调小。

#### 3.2.10 训练轮数 (Epochs)

- **配置项**：`训练轮数`
- **当前值**：`3.0`
- **详细解释**：
  - **作用**：指的是模型要把你的整个医疗数据集完整地“看”几遍。
  - **影响**：
    - **过少 (比如 1)**：模型可能还没学会，导致“欠拟合”。
    - **过多 (比如 10)**：模型可能会死记硬背训练题，导致“过拟合”，遇到新病历就不会治了。
  - **推荐**：`3.0` 是绝大多数 SFT 任务的标准起点，非常合适。

#### 3.2.11 最大样本数 (Max Samples) —— **重要警示**

- **配置项**：`最大样本数`
- **当前值**：`1000`
- **详细解释**：
  - **作用**：强行限制只使用数据集里的前 1000 条数据来训练。
  - **新手误区警报**：
    - **调试模式**：如果你只是想花几分钟测试一下 A10 显卡能不能跑通代码，设置 `1000` 是对的。
    - **正式训练**：**请务必清空此项**（留空）。如果你的医疗数据集有 10 万条数据，填了 `1000` 就意味着模型丢弃了 99% 的知识，这会导致训练出来的模型极其弱智。
  - **操作建议**：跑通测试后，正式跑数据时记得删掉这个数字。

#### 3.2.12 截断长度 (Cutoff Length)

- **配置项**：`截断长度`
- **当前值**：`2048`
- **详细解释**：
  - **作用**：限制一条数据（输入问题 + 模型回答）的最大总字数（Token 数）。超过的部分会被“一刀切”切掉。
  - **生产建议**： **Medical O1 Reasoning (医疗推理)**。这类数据通常包含很长的思维链（Chain of Thought），详细描述病情和推理过程。`2048` 可能偏短，容易把关键的推理结论切掉。

#### 3.2.13 显存控制核心：批处理与梯度累积

这里是决定你训练速度和显存占用的“油门”和“刹车”。

- **配置项**：`批处理大小 (Batch Size)`
- **当前值**：`1`
- **详细解释**：
  - **含义**：每个 GPU 一次读入并计算 1 条数据。
- **配置项**：`梯度累积 (Gradient Accumulation)`
- **当前值**：`4`
- **详细解释**：
  - **含义**：攒够 4 次（1x4=4）再修改一次模型参数。
  - **真实批大小**：实际生效的 Batch Size = `批处理大小` * `梯度累积` = 1 * 4 = 4。

#### 3.2.14 验证集比例 (Val Ratio)

- **配置项**：`验证集比例`
- **当前值**：`0.15` (15%)
- **详细解释**：
  - **作用**：从训练数据里切出 15% 的题不给模型学，专门留到考试用，看看模型是不是真的懂了（还是死记硬背）。
  - **建议**：
    - 如果数据总量很大（比如 >1万条），15% 可能太多了（浪费数据），可以改为 `0.05`。
    - 如果数据量少（<1000条），`0.15` 是合理的。

#### 3.2.15 计算类型 (Compute Type)

- **配置项**：`计算类型`
- **当前值**：`bf16`
- **详细解释**：
  - **含义**：使用 Brain Float 16 精度进行计算。
  - **环境匹配**：**完美**。你的 A10 显卡（Ampere 架构）原生支持 bf16。相比 fp16，bf16 更加稳定，训练过程中不容易出现 Loss 变成 NaN（数值溢出）的报错。请务必保持 `bf16`。

![image-20251204190718700](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041907846.png)

#### 3.2.16 日志与保存配置 (Logging & Checkpointing)

- **配置项**：`日志间隔`
- **当前值**：`5`
- **详细解释**：
  - **作用**：每训练 5 步（Steps），在屏幕上打印一次当前的 Loss（损失值）和学习率。
  - **建议**：保持默认。这个频率能让你实时看到模型是不是“学废了”（Loss 暴涨）或者“学得好”（Loss 稳步下降），方便及时终止错误训练。
- **配置项**：`保存间隔`
- **当前值**：`100`
- **详细解释**：
  - **作用**：每训练 100 步，自动把当前训练好的模型权重保存为一个文件夹（Checkpoint）。
  - **硬盘空间预警**：虽然 LoRA 文件很小（几十 MB），但如果你的训练总步数很多（比如 10000 步），每 100 步存一次会产生 100 个文件夹，管理起来很麻烦。
  - **优化建议**：如果是正式训练，建议改为 `500` 或 `1000`，避免生成太多中间文件。

#### 3.2.17 预热步数 (Warmup Steps)

- **配置项**：`预热步数`
- **当前值**：`0`
- **详细解释**：
  - **含义**：在训练刚开始时，先用很小的学习率慢慢热身，然后再增加到 `1e-4`。
  - **作用**：防止模型一开始因为“步子迈太大”而跑偏。
  - **建议**：对于医疗推理这种复杂任务，建议设置为总步数的 10% 左右（例如填 `10` 或 `20`）。这能让模型更平滑地进入学习状态。填 `0` 也能跑，但不够稳健。

#### 3.2.18 额外参数深度解析 (Extra Arguments) —— **核心优化**

输入框中填写的这段代码是整个配置的“点睛之笔”：

`{"optim": "adamw_torch", "gradient_checkpointing": true}`

这两个参数对 A10 显卡的影响极大，下面逐一解释：

**1. 优化器设置 (`"optim": "adamw_torch"`)**

- **含义**：指定使用 PyTorch 官方实现的 AdamW 优化器。
- **作用**：
  - **稳定性**：这是最标准、兼容性最好的优化器实现。相比于一些魔改版（如 8bit-adam），它更不容易报错。
  - **显存代价**：它需要的显存稍多一点点，但在你的 24G 显卡 + 4-bit 量化环境下，这点开销完全可以接受。

**2. 梯度检查点 (`"gradient_checkpointing": true`) —— 省显存神器**

- **含义**：这是一种“以时间换空间”的技术。
  - **不开启时**：模型在向前计算（Forward）时，会把每一层的中间结果（Activations）都存满显存，留着给反向传播（Backward）用。这非常吃显存！
  - **开启后**：模型不存中间结果了。等需要反向传播时，再临时重新算一遍。
- **对 A10 (24G) 的巨大价值**：
  - **显存暴降**：开启此选项通常能节省 **20%** 的显存占用。
  - **代价**：训练速度会变慢约 20-30%。
  - **结论**：**必须开启**。对于单卡训练，为了能跑长文本（医疗病历），牺牲一点速度换取更大的上下文空间是非常划算的。

#### 3.2.19 NEFTune 噪声参数

- **配置项**：`NEFTune 噪声参数`
- **当前值**：`0`
- **详细解释**：
  - **作用**：一种给 Embedding 层加噪声的技术，能防止过拟合。
  - **初学者建议**：保持 `0`（关闭）。目前这个技术还在实验阶段，有时候会导致模型变得“胡言乱语”。对于初次上手，我们先追求“跑通”和“稳定”。

![image-20251204191239259](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041912330.png)

#### 3.2.20 LoRA 秩 (LoRA Rank)

- **配置项**：`LoRA 秩`
- **当前值**：`8`
- **详细解释**：
  - **含义**：可以理解为 LoRA 这个“外挂大脑”的容量或带宽。数值越大，模型能学到的复杂知识越多，但生成的权重文件（Adapter）也越大，显存占用稍高。
  - **A10 24G 建议**：
    - `8` 是 LLaMA-Factory 的默认值，非常节省资源，适合入门跑通流程。
    - **进阶建议**：针对 **Medical O1 Reasoning** 这种需要强逻辑推理的任务，`8` 可能略显单薄。如果您的显存还很富裕（结合 4-bit 量化通常是富裕的），建议尝试将此值调高至 **`16`** 或 **`32`**，甚至 **`64`**。这能显著提升模型对复杂医疗逻辑的拟合能力。

#### 3.2.21 LoRA 缩放系数 (LoRA Alpha)

- **配置项**：`LoRA 缩放系数`
- **当前值**：`16`
- **详细解释**：
  - **含义**：决定了 LoRA 学到的新权重对原模型影响的“倍率”。
  - **黄金法则**：通常设置为 `LoRA 秩` 的 2 倍。
  - **操作建议**：如果您把上面的 Rank 改成了 16，记得把这里的 Alpha 改成 32；如果 Rank 是 32，这里就填 64。保持 `Alpha = 2 * Rank` 这个比例是最稳健的做法。

#### 3.2.22 LoRA+ 学习率比例 (LoRA+ LR Ratio) —— **高级技巧**

- **配置项**：`LoRA+ 学习率比例`
- **当前值**：`16`
- **详细解释**：
  - **含义**：这是一个名为 **LoRA+** 的优化算法。它让 LoRA 的两个矩阵（矩阵 A 和矩阵 B）使用不同的学习率进行训练。
  - **作用**：研究表明，让矩阵 B 学得比矩阵 A 快（通常快 16 倍左右），能大幅提升训练效率和效果。
  - **环境影响**：开启这个选项（只要数值不为 0 即开启）不会增加显存消耗，属于“免费的午餐”。对于 Qwen 系列模型，保留默认值 `16` 效果通常很好。

#### 3.2.23 LoRA 作用模块 (Target Modules) —— **关键配置**

- **配置项**：`LoRA 作用模块`
- **当前值**：`all`
- **详细解释**：
  - **含义**：指定 LoRA 具体要挂载在模型的哪些层上（例如 Q、K、V 注意力层，或者 MLP 层）。
  - **当前设置 "all"**：这是一个非常激进且优秀的设置。它意味着 LoRA 会挂载到模型所有的线性层上（`q_proj`, `k_proj`, `v_proj`, `o_proj`, `gate_proj`, `up_proj`, `down_proj`）。
  - **对医疗推理的影响**：
    - 仅挂载 Q/V（传统做法）：模型只能学到一点皮毛。
    - **挂载 "all"**：模型能全方位地调整自己的逻辑处理能力。对于推理类（Reasoning）任务，必须选 `all` 才能获得最佳效果。
  - **硬件压力**：相比只挂载 Q/V，选择 `all` 会增加一些显存消耗和计算量。但在 **A10 (24G)** 上，这完全在承受范围内。**请务必保持 "all"**。

![image-20251204191714495](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041917591.png)

### 3.3 启动微调
将输出目录修改为`train_qwen3_8b`，训练后的LoRA权重将会保存在此目录中。点击「预览命令」可展示所有已配置的参数，您如果想通过代码运行微调，可以复制这段命令，在命令行运行。

点击「开始」启动模型微调。
![image-20251203170030807](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031700144.png)

启动微调后需要等待一段时间，待模型下载完毕后可在界面观察到训练进度和损失曲线。模型微调大约需要50分钟，显示“训练完毕”代表微调成功。

![image-20251204200651649](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042006782.png)

#### 3.3.1 训练效率与稳定性深度解析 (Efficiency & Stability)

这部分主要评估您的硬件资源是否被充分利用，以及训练过程是否健康。

- **硬件适配度：满分**
  - **显存状态**：日志全称无 OOM (Out Of Memory) 报错，说明 `4-bit 量化` + `梯度检查点` 的策略非常成功。在 A10 (24G) 显卡上，显存占用被牢牢控制在安全线以内。
  - **计算精度**：日志显示 `compute dtype: torch.bfloat16`。A10 显卡原生支持 `bf16`，这比 `fp16` 更稳定，能有效防止训练中出现 Loss=NaN（数值溢出）的灾难性错误。
- **训练速度：极快**
  - **吞吐量**：稳定在 **620 - 630 tokens/s**。对于 8B 参数量的模型，在单卡上达到这个速度属于**第一梯队**的效率。
  - **样本处理**：约 `0.86 samples/s`。这意味着处理一条几千字的复杂医疗推理数据，只需要 1.2 秒左右。
- **结论**：您的环境配置（PyTorch 2.6 + CUDA 12.4 + A10）非常健壮，完全可以支撑后续更大规模的训练。

#### 3.3.2 损失函数与过拟合深度诊断 (Loss Analysis) —— **最关键的发现**
> 输出目录（在界面上显示的 train_2025-12-04-19-04-58），路径通常是： saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/
![image-20251204201640965](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042016031.png)
![image-20251204201557168](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042015285.png)

这是整个日志中最有价值的数据，它告诉我们模型到底学得怎么样。请重点关注 **Eval Loss (模拟考试成绩)** 的变化轨迹。

- **第一阶段：快速学习期 (Step 0 - 200)**
  - **表现**：Train Loss 迅速下降，Eval Loss 也从 `0.5516` 降至 **`0.5446`** (Step 200)。
  - **解读**：此时模型正在像海绵一样吸收知识，它学会了如何模仿医生的语气，也掌握了基本的推理逻辑。
  - **状态**：**最佳 (Optimal)**。
- **第二阶段：死记硬背期 (Step 200 - 400)**
  - **表现**：Train Loss 继续下降（平时作业全对），但 Eval Loss 开始反弹至 `0.57` - `0.60`。
  - **解读**：模型开始为了刷低训练集的 Loss，去死记硬背某些特定的病历描述，而不是理解其中的医理。它开始“偏科”了。
- **第三阶段：严重过拟合期 (Step 400 - 639)**
  - **表现**：Train Loss 降到了极低的 `0.21`，但 Eval Loss 飙升至 **`0.688`**，甚至比刚开始训练时还要差。
  - **解读**：模型彻底“学傻了”。如果你问它训练集里一模一样的问题，它能答得很完美；但如果你问一个新的病人情况，它可能会胡言乱语，因为它的通用能力被破坏了。

#### 3.3.3 学习内容与数据质量验证 (Data Quality Verification)

日志中有一段数据的 Input/Output 预览，这证实了您的数据清洗工作是成功的。

- **标签识别**：日志显示 `labels: <think>从中医的角度来看...</think>`。
- **深度解读**：
  - 模型不仅学到了“答案”，还学到了被 `<think>` 标签包裹的**“隐式思维链”**。
  - 这意味着在推理时，模型会先在内部进行一步步的逻辑推导（比如先分析病因，再定性，最后给方案），这正是 OpenAI o1 等推理模型的核心特性。

#### 3.3.4 最佳模型锁定 (Model Selection)

这是新手最容易犯错的地方：**千万不要以为跑完最后一步的模型就是最好的。**

- **当前陷阱**：默认输出的 `checkpoint-639` 是一个过拟合的模型，泛化能力很差。
- **正确选择**：请回溯到 **`checkpoint-200`**。
- **操作建议**：在 WebUI 中加载模型时，请手动选择 `checkpoint-200` 文件夹。如果可能，建议您把这个文件夹单独复制出来备份，重命名为 `Qwen3-Medical-Best`。

------

#### 3.3.5 核心指标总结表 (Summary Table)

为了方便您存档和对比，我为您总结了这份详细的训练报告表：

| **核心维度**   | **细分指标** | **结果数值/状态**      | **初学者解读与建议**                                         |
| -------------- | ------------ | ---------------------- | ------------------------------------------------------------ |
| **硬件与环境** | 显卡资源     | A10 (24G) 单卡         | **完美**。显存占用合理，无溢出，利用率高。                   |
|                | 计算精度     | bfloat16 (bf16)        | **优秀**。比 fp16 更稳定，适合 A10 架构。                    |
|                | 量化策略     | 4-bit (QLoRA)          | **关键**。这是实现 620+ tokens/s 高速训练的核心原因。        |
| **数据情况**   | 数据总量     | 850 条                 | **偏少**。数据越少，越容易过拟合（本次 Step 200 就过拟合了）。 |
|                | 数据格式     | CoT (思维链)           | **正确**。成功识别 `<think>` 标签，模型学会了推理步骤。      |
| **训练表现**   | 总耗时       | 49 分钟                | 效率很高，适合快速迭代实验。                                 |
|                | 最佳轮数     | Epoch ~0.94 (Step 200) | **重要知识点**：并不是轮数越多越好，对于小数据，1 轮往往就够了。 |
| **模型效果**   | 最佳 Loss    | **0.5446** (验证集)    | 出现在 **Step 200**。这是模型“智力”的巅峰时刻。              |
|                | 最终 Loss    | 0.6880 (验证集)        | 出现在结束时。说明模型后期在“负优化”。                       |
| **最终决策**   | **最佳存档** | **checkpoint-200**     | **请使用此版本！** 它是本次训练的精华。                      |
|                | **避坑存档** | checkpoint-639         | 已过拟合，建议仅作对比研究，不建议用于生产。                 |
| **后续优化**   | 策略调整     | 减少 Epoch / 增加数据  | 建议下次将 `Epoch` 设为 1，或者将数据扩充到 3000 条以上。    |

**一句话总结**：这是一次技术上非常成功的运行，环境与参数配置完美，但由于数据量较少（850条），模型在第 1 轮（Step 200）就达到了最佳状态，后续的训练属于过度学习。请使用 **Checkpoint-200** 作为最终成果。

## 4. 模型评估


微调完成后，点击页面顶部的「刷新适配器」，然后点击适配器路径，即可弹出刚刚训练完成的LoRA权重，点击选择下拉列表中的train_llama3选项，在模型启动时即可加载微调结果。
![image.png](./_html/evaluate_adaptor.jpg)

选择「Evaluate&Predict」栏，在数据集下拉列表中选择「eval」（验证集）评估模型。更改输出目录为`eval_llama3`，模型评估结果将会保存在该目录中。最后点击开始按钮启动模型评估。
![image.png](./_html/evaluate_start.jpg)

模型评估大约需要5分钟左右，评估完成后会在界面上显示验证集的分数。其中ROUGE分数衡量了模型输出答案（predict）和验证集中标准答案（label）的相似度，ROUGE分数越高代表模型学习得更好。
![image.png](./_html/evaluate_result.jpg)


## 5. 模型对话

选择「Chat」栏，确保适配器路径是`train_llama3`，点击「加载模型」即可在Web UI中和微调模型进行对话。
![image.png](./_html/chat_params.jpg)

在页面底部的对话框输入想要和模型对话的内容，点击「提交」即可发送消息。发送后模型会逐字生成回答，从回答中可以发现模型学习到了数据集中的内容，能够恰当地模仿诸葛亮的语气对话。
![image.png](./_html/chat_result1.jpg)

点击「卸载模型」，点击“×”号取消适配器路径，再次点击「加载模型」，即可与微调前的原始模型聊天。
![image.png](./_html/chat_uninstall.jpg)

重新向模型发送相同的内容，发现原始模型无法模仿诸葛亮的语气生成中文回答。
![image.png](./_html/chat_result2.jpg)


## 6. 总结

本次教程介绍了如何使用PAI和LLaMA Factory框架，基于轻量化LoRA方法微调Qwen3模型，使其能够进行中文问答和角色扮演，同时通过验证集ROUGE分数和人工测试验证了微调的效果。在后续实践中，可以使用实际业务数据集，对模型进行微调，得到能够解决实际业务场景问题的本地领域大模型。